# problema di trasporto 2

Due depositi (A e B) di carburante riforniscono tre distributori (1,2 e 3).

Ogni deposito ha una disponibilità limitata di carburante e ogni distributore ha una richiesta nota.

I costi di trasporto tra depositi e distributori dipende dalla distanza e dalla quantità di carburante

![img](./img.png)

![tab](./tab.png)

Quanto carburante inviare da ogni deposito a ogni distributore in modo da soddisfare tutte le richieste al costo minimo?


## formulazione generale

Una rete logistica (single-commodity) è formata da n sorgenti $S_1$,...,$S_n$ e $m$ destinazioni $T_1$ ,...,$T_m$.

Dati la disponibilità di prodotto di ogni sorgente, la richiesta di prodotto di ogni destinazione e il costo unitario di trasporto per ogni coppia sorgente-destinazione, quanto prodotto trasportare da ogni sorgente a ogni destinazione in modo da rispettare le disponibilità delle sorgenti e soddisfare le richieste delle destinazioni al costo minimo?

## modello parametrico

![param](./param.png)

![param2](./param2.png)

In [ ]:
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition

model = ConcreteModel()
model.name = "problema di miscelazione"

i = [ 'deposito A', 'deposito B' ]
d = [ 1000, 1400 ]

j = [ 'milano', 'roma', 'napoli' ]
b = [ 800, 700, 900 ]

a = [[13,   11,  16], 
     [12,   15,  14]]